# Exploring LakeFS with PySpark

This uses the [Everything Bagel](https://github.com/treeverse/lakeFS/tree/master/deployments/compose) Docker Compose environment.

[@rmoff](https://twitter.com/rmoff/) 

## Setup

### Install libraries

(could be built into the `Dockerfile`)

In [1]:
import sys
!{sys.executable} -m pip install lakefs_client

In [2]:
import sys
print("Kernel:", sys.executable)
print("Python version:", sys.version)

import pyspark
print("PySpark version:", pyspark.__version__)


Kernel: /opt/conda/bin/python
Python version: 3.9.7 | packaged by conda-forge | (default, Oct 10 2021, 15:08:54) 
[GCC 9.4.0]
PySpark version: 3.2.0


###  Spark

_With the necessary Delta Lake config too_

In [28]:
from pyspark import SparkFiles
from pyspark.sql.session import SparkSession

spark = (
    SparkSession.builder.master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore")
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.endpoint", "http://lakefs:8000")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.access.key", "AKIA-EXAMPLE-KEY")
    .config("spark.hadoop.fs.s3a.secret.key", "EXAMPLE-SECRET")    
    .getOrCreate()
)        

#### Test delta - write/read local

In [22]:
data = spark.range(0, 5)
data.write.format("delta").mode("overwrite").save("/tmp/delta-table")

In [23]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  1|
|  0|
|  3|
|  2|
|  4|
+---+



#### Test delta - write/read lakeFS

In [29]:
data = spark.range(0, 5)
df.write.format("delta").mode('overwrite').save('s3a://example/main/test')

Py4JJavaError: An error occurred while calling o172.save.
: java.util.concurrent.ExecutionException: java.nio.file.AccessDeniedException: s3a://example/main/test/_delta_log: getFileStatus on s3a://example/main/test/_delta_log: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: null; S3 Extended Request ID: null; Proxy: null), S3 Extended Request ID: null:403 Forbidden
	at com.google.common.util.concurrent.AbstractFuture$Sync.getValue(AbstractFuture.java:306)
	at com.google.common.util.concurrent.AbstractFuture$Sync.get(AbstractFuture.java:293)
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:116)
	at com.google.common.util.concurrent.Uninterruptibles.getUninterruptibly(Uninterruptibles.java:135)
	at com.google.common.cache.LocalCache$Segment.getAndRecordStats(LocalCache.java:2410)
	at com.google.common.cache.LocalCache$Segment.loadSync(LocalCache.java:2380)
	at com.google

In [20]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  4|
|  2|
|  3|
|  0|
|  1|
+---+



### LakeFS

In [25]:
import lakefs_client
from lakefs_client import models
from lakefs_client.client import LakeFSClient
from lakefs_client.api import branches_api
from lakefs_client.api import commits_api

# lakeFS credentials and endpoint
configuration = lakefs_client.Configuration()
configuration.username = 'AKIA-EXAMPLE-KEY'
configuration.password = 'EXAMPLE-SECRET'
configuration.host = 'http://lakefs:8000'

client = LakeFSClient(configuration)
api_client = lakefs_client.ApiClient(configuration)

#### List the current branches in the repository

https://pydocs.lakefs.io/docs/BranchesApi.html#list_branches

In [26]:
repo='example'

In [27]:
for b in client.branches.list_branches(repo).results:
    display(b.id)

'main'

## Load some data into lakeFS

Read a parquet file from URL

In [14]:
# The sample parquet file is Apache 2.0 licensed so perhaps include it in the Everything Bagel distribution? 
url='https://github.com/Teradata/kylo/blob/master/samples/sample-data/parquet/userdata1.parquet?raw=true'
spark.sparkContext.addFile(url)
df = spark.read.parquet("file://" + SparkFiles.get("userdata1.parquet"))

How many rows of data?

In [15]:
display(df.count())

1000

What does the data look like?

In [16]:
display(df.show(n=1,vertical=True))

-RECORD 0--------------------------------
 registration_dttm | 2016-02-03 07:55:29 
 id                | 1                   
 first_name        | Amanda              
 last_name         | Jordan              
 email             | ajordan0@com.com    
 gender            | Female              
 ip_address        | 1.197.201.2         
 cc                | 6759521864920116    
 country           | Indonesia           
 birthdate         | 3/8/1971            
 salary            | 49756.53            
 title             | Internal Auditor    
 comments          | 1E+02               
only showing top 1 row



None

## Write data to lakeFS (on the `main` branch) in Delta format

N.B. the connection to s3a is configured in the Docker Compose's `./etc/hive-site.xml` file. 

In [17]:
branch='main'

In [18]:
df.write.format("delta").mode('overwrite').save('s3a://'+repo+'/'+branch+'/demo/users')

Py4JJavaError: An error occurred while calling o96.save.
: java.util.concurrent.ExecutionException: java.nio.file.AccessDeniedException: s3a://example/main/demo/users/_delta_log: getFileStatus on s3a://example/main/demo/users/_delta_log: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: null; S3 Extended Request ID: null; Proxy: null), S3 Extended Request ID: null:403 Forbidden
	at com.google.common.util.concurrent.AbstractFuture$Sync.getValue(AbstractFuture.java:306)
	at com.google.common.util.concurrent.AbstractFuture$Sync.get(AbstractFuture.java:293)
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:116)
	at com.google.common.util.concurrent.Uninterruptibles.getUninterruptibly(Uninterruptibles.java:135)
	at com.google.common.cache.LocalCache$Segment.getAndRecordStats(LocalCache.java:2410)
	at com.google.common.cache.LocalCache$Segment.loadSync(LocalCache.java:2380)
	at

### The data as seen from LakeFS

https://pydocs.lakefs.io/docs/ObjectsApi.html#list_objects

Note the `physical_address` and its match in the S3 output in the next step

In [ ]:
client.objects.list_objects(repo,branch).results

In [ ]:
data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta-table")

### The data as seen from S3

In [ ]:
from pyspark.context import SparkContext
from pyspark import SparkFiles
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [ ]:
for o in s3.Bucket(repo).objects.all():
    print(o.last_modified, o.key, o.size)

### List diff of branch in LakeFS (this is kinda like a `git status`)

https://pydocs.lakefs.io/docs/BranchesApi.html#diff_branch

_Note that the files show **`'type': 'added'`**_

In [ ]:
api_instance = branches_api.BranchesApi(api_client)

api_response = api_instance.diff_branch(repo, branch)
if api_response.pagination.results==0:
    display("Nothing to commit")
else:
    for r in api_response.results:
        display(r)

### Commit the new file in `main`

https://pydocs.lakefs.io/docs/CommitsApi.html#commit

In [ ]:
from lakefs_client.api import commits_api
from lakefs_client.model.commit import Commit
from lakefs_client.model.commit_creation import CommitCreation

api_instance = commits_api.CommitsApi(api_client)
commit_creation = CommitCreation(
    message="Everything Bagel - commit users data (original)",
    metadata={
        "foo": "bar",
    }
) 

api_instance.commit(repo, branch, commit_creation)

### List branch status again - nothing returned means that there is nothing uncommitted

In [ ]:
api_instance = branches_api.BranchesApi(api_client)

api_response = api_instance.diff_branch(repo, branch)
if api_response.pagination.results==0:
    display("Nothing to commit")
else:
    for r in api_response.results:
        display(r)

_Similar to a `git status` showing `Your branch is up to date with 'main'` / `nothing to commit, working tree clean`_

## Create a branch

https://pydocs.lakefs.io/docs/BranchesApi.html#create_branch

**TODO** Show that there's no additional object created on object store (http://localhost:9001/buckets/example/browse login `minioadmin`/`minioadmin`)

In [ ]:
branch='add_more_user_data'

In [ ]:
from lakefs_client.model.branch_creation import BranchCreation

api_instance = branches_api.BranchesApi(api_client)
branch_creation = BranchCreation(
    name=branch,
    source="main",
) 

api_response = api_instance.create_branch(repo, branch_creation)
display(api_response)

### List the current branches in the `example` repository

https://pydocs.lakefs.io/docs/BranchesApi.html#list_branches

In [ ]:
for b in client.branches.list_branches(repo).results:
    display(b.id)

## Confirm that you can see the same data on the new branch

In [ ]:
xform_df = spark.read.parquet('s3a://'+repo+'/'+branch+'/demo/users')

How many rows of data?

In [ ]:
display(xform_df.count())

What does the data look like?

In [ ]:
display(xform_df.show(n=1,vertical=True))

### Note that on S3 there is still just the original 78k object - we've not duplicated any data for the new branch

In [ ]:
for o in s3.Bucket(repo).objects.all():
    print(o.last_modified, o.key, o.size)

## Add some new data

In [ ]:
# The sample parquet file is Apache 2.0 licensed so perhaps include it in the Everything Bagel distribution? 
url='https://github.com/Teradata/kylo/blob/master/samples/sample-data/parquet/userdata2.parquet?raw=true'
sc.addFile(url)
df = spark.read.parquet("file://" + SparkFiles.get("userdata2.parquet"))

In [ ]:
df.show(n=1,vertical=True)

## Write the data to the new branch and commit it

In [ ]:
df.write.mode('append').parquet('s3a://'+repo+'/'+branch+'/demo/users')

LakeFS sees that there is an uncommited change

In [ ]:
api_instance = branches_api.BranchesApi(api_client)

api_response = api_instance.diff_branch(repo, branch)
if api_response.pagination.results==0:
    display("Nothing to commit")
else:
    for r in api_response.results:
        display(r)

Commit it

In [ ]:
from lakefs_client.api import commits_api
from lakefs_client.model.commit import Commit
from lakefs_client.model.commit_creation import CommitCreation

api_instance = commits_api.CommitsApi(api_client)
commit_creation = CommitCreation(
    message="Everything Bagel - add more user data",
    metadata={
        "foo": "bar",
    }
) 

api_instance.commit(repo, branch, commit_creation)

## Re-read `main` and `add_more_user_data` branches and count rows

Original branch (`main`):

In [ ]:
main = spark.read.parquet('s3a://'+repo+'/main/demo/users')
display(main.count())

New branch (`add_more_user_data`):

In [ ]:
add_more_user_data = spark.read.parquet('s3a://'+repo+'/add_more_user_data/demo/users')
display(add_more_user_data.count())

### Look at the view in LakeFS

#### `main`

In [ ]:
client.objects.list_objects(repo,'main').results

#### `add_more_user_data`

In [ ]:
client.objects.list_objects(repo,'add_more_user_data').results

### The data as seen from S3

Note that there are just two 78k files; there is no duplication of data shared by branches.

In [ ]:
for o in s3.Bucket(repo).objects.all():
    print(o.last_modified, o.key, o.size)

## Create a new branch and test removing some data

In [ ]:
branch='remove_pii'

In [ ]:
from lakefs_client.model.branch_creation import BranchCreation

api_instance = branches_api.BranchesApi(api_client)
branch_creation = BranchCreation(
    name=branch,
    source="main",
) 

api_response = api_instance.create_branch(repo, branch_creation)
display(api_response)

### List the current branches in the `example` repository

https://pydocs.lakefs.io/docs/BranchesApi.html#list_branches

In [ ]:
for b in client.branches.list_branches(repo).results:
    display(b.id)

### Confirm that you can see the same data on the new branch

In [ ]:
xform_df = spark.read.parquet('s3a://'+repo+'/'+branch+'/demo/users')

How many rows of data? 

_Note that this shows 1000 per `main`, and not 2000 per the `add_more_user_data` branch above since this has not been merged to `main`_

In [ ]:
display(xform_df.count())

If you are reading and write a file from the same place, you need to use `.cache()` otherwise the write will fail with an error like this: 

```
Caused by: java.io.FileNotFoundException: 
No such file or directory: s3a://example/remove_pii/demo/users/part-00000-7a0bbe79-a3e2-4355-984e-bd8b950a4e0c-c000.snappy.parquet
```

[solution src](https://stackoverflow.com/a/65330116/350613)

### Transform the data

In [ ]:
df2=xform_df.drop('ip_address','birthdate','salary','email').cache()
# You need to do something to access the DF otherwise the `cache()` won't have any effect
df2.show(n=1,vertical=True)

### Write data back to the branch

In [ ]:
df2.write.mode('overwrite').parquet('s3a://'+repo+'/'+branch+'/demo/users')

### Commit changes

In [ ]:
api_instance = commits_api.CommitsApi(api_client)
commit_creation = CommitCreation(
    message="Remove PII",
) 

api_instance.commit(repo, branch, commit_creation)

## Re-read all branches and inspect data for isolation

Original branch (`main`):

In [ ]:
main = spark.read.parquet('s3a://'+repo+'/main/demo/users')
display(main.count())
main.show(n=1,vertical=True)

New branch (`add_more_user_data`):

In [ ]:
add_more_user_data = spark.read.parquet('s3a://'+repo+'/add_more_user_data/demo/users')
display(add_more_user_data.count())
add_more_user_data.show(n=1,vertical=True)

New branch (`remove_pii`):

In [ ]:
remove_pii = spark.read.parquet('s3a://'+repo+'/remove_pii/demo/users')
display(remove_pii.count())
remove_pii.show(n=1,vertical=True)

### Look at the view in LakeFS

#### `main`

In [ ]:
client.objects.list_objects(repo,'main').results

#### `add_more_user_data`

In [ ]:
client.objects.list_objects(repo,'add_more_user_data').results

#### `remove_pii`

In [ ]:
client.objects.list_objects(repo,'remove_pii').results

## Merge `remove_pii` into `main`

In [ ]:
client.refs.merge_into_branch(repository=repo, source_ref='remove_pii', destination_branch='main')

Original branch (`main`):

In [ ]:
main = spark.read.parquet('s3a://'+repo+'/main/demo/users')
display(main.count())
main.show(n=1,vertical=True)